In [35]:
with open('input.txt', 'r') as f:
    instructions = f.read().strip().split('\n')
__import__('pprint').pprint(instructions)


def is_integer(expr):
    try:
        int(expr)
        return True
    except ValueError:
        return False
    
def read_instructions(instructions, reg={'a': 0, 'b': 0, 'c': 0, 'd': 0}):
    to_value = lambda expr: int(expr) if is_integer(expr) else reg[expr]
    
    i = 0
    while i < len(instructions):
        line = instructions[i]
        cmd, *args = line.split()
        
        if cmd == 'tgl':
            target = i + to_value(args[0])
            if not (0 <= target < len(instructions)): 
                i += 1
                continue
            target_cmd, *target_args = instructions[target].split()
            
            if len(target_args) == 1:
                tgl_result = 'inc'
                if target_cmd == 'inc':
                    tgl_result = 'dec'
            else:
                tgl_result = 'jnz'
                if target_cmd == 'jnz':
                    tgl_result = 'cpy'
            instructions[target] = tgl_result + ' ' + ' '.join(target_args)
            
        elif cmd == 'inc':
            if args[0] in reg:
                reg[args[0]] += 1
        elif cmd == 'dec':
            if args[0] in reg:
                reg[args[0]] -= 1
        elif cmd == 'cpy':
            if args[1] in reg:
                origin = to_value(args[0])
                reg[args[1]] = origin
        else:  # jnz
            check_value = to_value(args[0])
            if check_value:
                i += int(to_value(args[1]))
                continue
        i += 1
    return reg
read_instructions(instructions, reg={'a': 12, 'b': 0, 'c': 0, 'd': 0})

['cpy a b',
 'dec b',
 'cpy a d',
 'cpy 0 a',
 'cpy b c',
 'inc a',
 'dec c',
 'jnz c -2',
 'dec d',
 'jnz d -5',
 'dec b',
 'cpy b c',
 'cpy c d',
 'dec d',
 'inc c',
 'jnz d -2',
 'tgl c',
 'cpy -16 c',
 'jnz 1 c',
 'cpy 77 c',
 'jnz 73 d',
 'inc a',
 'inc d',
 'jnz d -2',
 'inc c',
 'jnz c -5']


KeyboardInterrupt: 

In [28]:
'-2'.isnumeric()

False

In [31]:
is_integer('-a')

False

# Stolen

In [39]:
with open('input.txt', 'r') as f:
    glines = f.read().strip().split('\n')


def is_num(s):
    try:
        int(s)
    except:
        return False
    return True

def getval(regs, x):
    if is_num(x):
        return int(x)
    else:
        return regs[x]

def toggle(tline):
    sp = tline.split(" ")

    instr = sp[0]
    if instr == "inc":
        return " ".join(["dec"] + sp[1:])
    elif instr == "dec" or instr == "tgl":
        return " ".join(["inc"] + sp[1:])
    elif instr == "jnz":
        return " ".join(["cpy"] + sp[1:])
    elif instr == "cpy":
        return " ".join(["jnz"] + sp[1:])
    else:
        assert False

def run(lines, part2=False):
    pc = 0
    regs = {"a": 7, "b": 0, "c": 0, "d": 0}
    if part2:
        regs["a"] = 12

    while True:
        if pc >= len(lines):
            break
        line = lines[pc]

        # Instruction tracing to find the loop to optimize:
        #print pc, line

        a, b = line.split(" ", 1)

        if a == "cpy":
            b, c = b.split(" ")
            b = getval(regs, b)
            if c in regs:
                regs[c] = b
            else:
                print("invalid")
            pc += 1
        elif a == "inc":
            if b in regs:
                # Peephole optimize inc/dec/jnz loops

                # 4 cpy b c
                # 5 inc a          <<<< 0
                # 6 dec c          <<<< +1
                # 7 jnz c -2       <<<< +2
                # 8 dec d          <<<< +3
                # 9 jnz d -5       <<<< +4

                if pc + 3 < len(lines) and pc - 1 >= 0 and lines[pc-1].startswith("cpy ") and \
                    lines[pc+1].startswith("dec") and lines[pc+2].startswith("jnz") and \
                    lines[pc+3].startswith("dec") and lines[pc+4].startswith("jnz"):

                    incop = b

                    cpysrc, cpydest = lines[pc-1].split(" ")[1:]
                    dec1op = lines[pc+1].split(" ")[1]
                    jnz1cond, jnz1off = lines[pc+2].split(" ")[1:]
                    dec2op = lines[pc+3].split(" ")[1]
                    jnz2cond, jnz2off = lines[pc+4].split(" ")[1:]

                    if cpydest == dec1op and dec1op == jnz1cond and \
                        dec2op == jnz2cond and \
                        jnz1off == "-2" and jnz2off == "-5":
                            # inner loop:
                            # incop += cpysrc
                            # dec1op <- 0

                            # outer loop:
                            # dec2op times

                            # net result:  incop += (cpysrc * dec2op)
                            # dec1op, dec2op <- 0
                            regs[incop] += (getval(regs, cpysrc) * getval(regs, dec2op))
                            regs[dec1op] = 0
                            regs[dec2op] = 0
                            pc += 5
                            continue


                regs[b] += 1
            pc += 1
        elif a == "dec":
            if b in regs:
                regs[b] -= 1
            pc += 1
        elif a == "jnz":
            b, c = b.split(" ")
            b = getval(regs, b)
            c = getval(regs, c)
            if b != 0:
                pc = pc + int(c)
            else:
                pc += 1
        elif a == "tgl":
            xr = b
            x = getval(regs, xr)

            iidx = pc + x
            #print "tgl", x, iidx

            if iidx >= 0 and iidx < len(lines):
                tline = lines[iidx]
                lines[iidx] = toggle(tline)
                #print tline, "->", lines[iidx]
            pc += 1
        else:
            assert False

    return regs["a"]


print("part 1:", run(glines[:]))
print("part 2:", run(glines[:], part2=True))

part 1: 10661
part 2: 479007221
